<a href="https://colab.research.google.com/github/pastured3ton/Robot/blob/main/notebooks/inverse_kinematics_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
import google.colab
%pip install roboticstoolbox-python>=1.0.2
%pip install colored==1.4.4
%pip install numpy==1.26.4
%pip install matplotlib==3.7

In [199]:
import numpy as np
import roboticstoolbox as rtb
from spatialmath import *
from math import pi
import matplotlib.pyplot as plt
from matplotlib import cm
np.set_printoptions(linewidth=100, formatter={'float': lambda x: f"{x:8.4g}" if abs(x) > 1e-10 else f"{0:8.4g}"})

%matplotlib notebook

csv_rows = []
time = 0.0
dt = 0.1   # seconds between poses (adjust speed here)

The solution of the inverse kinematics problem, similarly to the forward kinematics, must be started with the creation of a model of the manipulator.

In [200]:
L1 = 0.0
L2 = 0.5
L3 = 0.3
L4 = 0.01

# Joint 1: Prismatic (sigma=1)
J1 = rtb.DHLink(d=0.0, alpha=0.0, a=L1, theta=0.0, sigma=1, qlim=[0, 1])

# Joint 2: Revolute
J2 = rtb.DHLink(d=0.0, alpha=0.0, a=L2, theta=0.0, sigma=0, qlim=[0, 4*pi])

# Joint 3: Revolute
J3 = rtb.DHLink(d=0.0, alpha=0.0, a=L3, theta=0.0, sigma=0, qlim=[0, 4*pi])

# Joint 4: Revolute
J4 = rtb.DHLink(d=0.0, alpha=0.0, a=L4, theta=0.0, sigma=0, qlim=[0, 4*pi])

# Build robot model
robot = rtb.DHRobot([J1, J2, J3, J4], name="Planar_TRRR")

In the next step, it is necessary to determine the position and orientation of the manipulator tip for which the problem is to be solved. This position and orientation should be presented in the form of a homogeneous matrix. One of the simplest ways is to define a translation and rotation matrix. The translation matrix is created using the **SE3** command as in the example:
```py
trans = SE3(0.1, 0.2, 0.3)
```
The subsequent arguments correspond to the x, y and z coordinates of the given point.

In [233]:
trans1 = SE3(0.450, 0.250, 0.3)
trans2 = SE3(0.375, 0.380, 0.3)
trans3 = SE3(0.225, 0.380, 0.3)
trans4 = SE3(0.150, 0.250, 0.3)
trans5 = SE3(0.225, 0.120, 0.3)
trans6 = SE3(0.375, 0.120, 0.3)
trans7 = SE3(0.450, 0.250, 0.3)

This creates a homogeneous matrix, the rotation part of which is an identity matrix.
To create the appropriate rotation matrix, use the command **SE3.OA**.
```py
y = [0,0,1]
z = [1,0,0]
rot = SE3.OA(y, z)
```
Parameters y and z refers to:
* y - vector parallel to the y axis of the tool
* z - vector parallel to the z axis of the tool

In [234]:
y = [1,0,0]
z = [0,0,1]
rot = SE3.OA(y, z)

Note that vectors y and z cannot be zero or parallel. However, it is not necessary to normalize the vectors or ensure their perpendicularity. In the case of a pair of non-perpendicular vectors, vector z will be kept in the resulting matrix and vector y will be fitted to it. This operation results from the fact that the vector z determines the so-called direction of approach, or simply the position of the main axis of the tool, the y vector is responsible for the rotation of the tool around this axis.

By multiplying the translation and rotation matrices obtained in that way, one can easily create the desired homogeneous transformation matrix.

In [235]:
T1 = trans1
T2 = trans2
T3 = trans3
T4 = trans4
T5 = trans5
T6 = trans6
T7 = trans7

The solution to the inverse kinematics problem can be obtained by calling the **ikine_LM** method on the robot object and passing a homogeneous matrix as an argument.
```py
sol = robot.ikine_LM(T)
```
The returned object *sol* contains solution of the problem: vector of joint coordinates and information about whether the given position was reached.

In [236]:
sol1 = robot.ikine_LM(T1)
sol2 = robot.ikine_LM(T2)
sol3 = robot.ikine_LM(T3)
sol4 = robot.ikine_LM(T4)
sol5 = robot.ikine_LM(T5)
sol6 = robot.ikine_LM(T6)
sol7 = robot.ikine_LM(T7)


In the example above, you can see that the calculated joint coordinates vector is \[1.074, 0.5266\]. However, the parameter success=False means that the given position has not been reached (i.e., there is no solution for inverse kinematics problem).

In [237]:
print(sol1.success)
print(sol1.q)
print(sol2.success)
print(sol2.q)
print(sol3.success)
print(sol3.q)
print(sol4.success)
print(sol4.q)
print(sol5.success)
print(sol5.q)
print(sol6.success)
print(sol6.q)
print(sol7.success)
print(sol7.q)

True
[     0.3 -0.08878    1.854   -1.765]
True
[     0.3    1.396    -1.78    0.384]
True
[     0.3    1.694   -2.092   0.3977]
True
[     0.3    0.496    2.605   -3.101]
True
[     0.3  0.04905    2.769   -2.818]
True
[     0.3  -0.3248    2.267   -1.942]
True
[     0.3 -0.08878    1.854   -1.765]


In this case, the problem with the solution is due to the very simple structure of the manipulator, which gives very limited mobility. In such cases, it may not be possible or necessary to maintain all position/orientation constraints. The tool anticipates such situations and makes it possible to specify in the **ikine_LM** method a mask responsible for which elements of the given position must be exactly reached. The mask should be a six-element array of 0s and 1s, in which successive elements mean the need to accurately reproduce, respectively: x, y, and z positions and rotations around the x, y and z axes.
```py
mask = np.array([0, 1, 1, 0, 0, 0])   # it is required to exactly reach y and z coordinates
sol = robot.ikine_LM(T, mask=mask)
```
You have to remember that the number of 1s in the mask cannot be larger than the number of degrees of freedom of the manipulator.

In [238]:
mask = np.array([1, 1, 1, 1, 1, 1])
sol1 = robot.ikine_LM(T1, mask=mask)
sol2 = robot.ikine_LM(T2, mask=mask)
sol3 = robot.ikine_LM(T3, mask=mask)
sol4 = robot.ikine_LM(T4, mask=mask)
sol5 = robot.ikine_LM(T5, mask=mask)
sol6 = robot.ikine_LM(T6, mask=mask)
sol7 = robot.ikine_LM(T7, mask=mask)

In [239]:
print(sol1.success)
print(sol1.q)
print(sol2.success)
print(sol2.q)
print(sol3.success)
print(sol3.q)
print(sol4.success)
print(sol4.q)
print(sol5.success)
print(sol5.q)
print(sol6.success)
print(sol6.q)
print(sol7.success)
print(sol7.q)


True
[     0.3    1.122   -1.854   0.7321]
True
[     0.3   0.2149     1.78   -1.995]
True
[     0.3   0.4175    2.092    -2.51]
True
[     0.3    1.625   -2.605   0.9808]
True
[     0.3  0.04905    2.769   -2.818]
True
[     0.3  -0.3248    2.267   -1.942]
True
[     0.3 -0.08878    1.854   -1.765]


Often, specifying only the given position may not be sufficient to find a solution, even though the position is reachable. If it is possible, it is also worth providing the initial value of the joint variables to the **ikine_LM** method.
```py
mask = np.array([0, 1, 1, 0, 0, 0])
q0 = np.array([0.0, 1.0])
sol = robot.ikine_LM(T, q0=q0, mask=mask)

```

In [240]:
mask = np.array([1, 1, 1, 1, 1, 1])
q0 = np.array([0.0, 1.0, 0.0, 0.0]) # Changed q0 to match the 4 joints of the robot
sol1 = robot.ikine_LM(T1, q0=q0, mask=mask)
q1=sol1
sol2 = robot.ikine_LM(T2, q0=q1, mask=mask)
q2=sol2
sol3 = robot.ikine_LM(T3, q0=q2, mask=mask)
q3=sol3
sol4 = robot.ikine_LM(T4, q0=q3, mask=mask)
q4=sol4
sol5 = robot.ikine_LM(T5, q0=q4, mask=mask)
q5=sol5
sol6 = robot.ikine_LM(T6, q0=q5, mask=mask)
q6=sol6
sol7 = robot.ikine_LM(T7, q0=q6, mask=mask)

print(sol1.success)
print(sol1.q)
print(sol2.success)
print(sol2.q)
print(sol3.success)
print(sol3.q)
print(sol4.success)
print(sol4.q)
print(sol5.success)
print(sol5.q)
print(sol6.success)
print(sol6.q)
print(sol7.success)
print(sol7.q)


True
[     0.3    1.122   -1.854   0.7321]
True
[     0.3    1.396    -1.78    0.384]
True
[     0.3    1.694   -2.092   0.3977]
True
[     0.3    1.625   -2.605   0.9808]
True
[     0.3   0.9691   -2.769      1.8]
True
[     0.3   0.9601   -2.267    1.307]
True
[     0.3    1.122   -1.854   0.7321]
